# LSDL CUB, Homework 4. Model size reduction [10 pts]

__Soft deadline 29.11.24 23:59__ \
__Hard deadline 2.12.24 23:59__

### About this task

In this task you will learn how to solve the Named Entity Recognition (NER) problem on the most popular dataset - [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). You will have a pre-trained BERT at your disposal, which you need to reduce with minimal loss in quality to the size of 20M parameters. To do this you will implement embedding factorisation, knowledge distillation, parameter sharing and so on.

In this assignment you will have to do quite a lot of experiments, so we recommend you not to write all the code in a notebook, but to make different files for separate logical blocks and compose everything in the form of a project. This will keep your notebook from getting too messy and will make the task much easier for both you and the graders. Also try to log all your experiments in wandb so that nothing gets lost.

### Grading

The grade for this homework assignment will be made up of a grade for __tasks__ and a __report__, in which, as before, you are required to write about the work you have done. You may receive up to 2 points for the report, but you may lose points for the tasks themselves if you do not write a report. The assignments are divided into two parts: _numerical_ and _of choice_. For _numerical_ you can get a total of 6 points, for _of choice_ assignments you can get up to 16 points. That is, you can get 24 points for the assignment. Anything you score above 10 will be considered bonuses.

### Dataset

Named Entity Recognition is the task of classifying tokens into entity classes. CoNLL-2003 uses **BIO** (Beggining, Inside, Outside) tagging to name entities, where the tags mean the following:

- *B-{entity}* - the beginning of the entity *{entity}*
- *I-{entity}* - the continuation of the entity *{entity}*
- *O* - not an entity.

There are other ways of labelling, such as BILUO. You can read about them [here](https://en.wikipedia.org/wiki/Inside-outside-beginning_(tagging)).

There are a total of 9 different tags in the dataset.
- O - no entity corresponds to the word.
- B-PER/I-PER - the word or set of words corresponds to a particular _person_.
- B-ORG/I-ORG - the word or set of words corresponds to a specific _organisation_.
- B-LOC/I-LOC - a word or set of words corresponds to a particular _location_.
- B-MISC/I-MISC - a word or set of words corresponds to an entity that does not belong to any of the previous ones. For example, a nationality, a work of art, an event, etc.

Let's start with loading and preprocessing the dataset.

In [1]:
from datasets import load_dataset

dataset = load_dataset("eriktks/conll2003")

dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3453
    })
})

In [2]:
dataset['train'][0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [3]:
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [4]:
words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]

for i in range(len(words)):
    print(f'{words[i]}\t{label_names[labels[i]]}')

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


### Preprocessing

Throughout this homework we will be using the _cased_ version of BERT, meaning that the tokeniser will take case of words into account. For the NER task, case is important because names and organisations or art objects are often capitalised and it would be silly to hide such information from the model.

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

During tokenisation, words can be split into several tokens (like the word `Fischler` from the example below), causing a mismatch between the number of tokens and labels. We will have to fix this mismatch manually.

In [6]:
example = dataset["train"][12]
words = example["tokens"]
tags = [label_names[t] for t in example["ner_tags"]]
tokenized_text = tokenizer(example["tokens"], is_split_into_words=True)

print('Words: ', words)
print('Tokens:', tokenized_text.tokens())
print('Tags:', tags)

Words:  ['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
Tokens: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Tags: ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']


__Task 1 [1 pts].__ Tokenise the entire dataset and for each text, align tokens with labels so that each token corresponds to one label. It is important to preserve the BIO notation while doing this. And don't forget the special tokens! You should get something like this:

In [9]:
aligned_labels = align_labels_with_tokens(example["ner_tags"], tokenized_text)
tags = [label_names[t] if t > -1 else t for t in aligned_labels]
print("Aligned tags:", aligned_labels)
print("Aligned named tags:", tags)

Aligned tags: [-100    0    5    0    5    0    1    2    2    0    0    0    0 -100]
Aligned named tags: [-100, 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', -100]


In [ ]:
# your code here

### Metric.

The F1 measure with micro-averaging is commonly used to evaluate the quality of NER. We will load it from the `seqeval` library. The `f1_score` function takes two 2d lists with correct and predicted labels written in text, and returns an F1 value for them. You can use it with default parameters.

In [12]:
# ! pip install seqeval

In [46]:
from seqeval.metrics import f1_score

A peculiarity of F1 metric for NER is that in some situations incorrect answers can be counted as correct answers. For example, if the model predicted `[‘I-PER’, ‘I-PER’]`, we can guess that it should actually be `[‘B-PER’, ‘I-PER’]`, since an entity cannot start with `I-`. The `f1_score` function takes this into account and therefore only works with textual representations of labels.

### Model

We will take `bert-base-cased` as our base model. As you realise, it has not been trained on the NER task. Therefore, it needs to be fine-tuned before proceeding to reduce its size.

__Task 2 [1 pts]__ Fine-tune `bert-base-cased` on our dataset using standard fine-tuning. You should get at least 0.9 F1 on the test set. Note that the higher the quality of the large model, the better the distilled student will perform. You can use the `Trainer` from Hugging Face for training.

In [10]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_names))

print('Number of parameters:', sum(p.numel() for p in model.parameters()))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of parameters: 107726601


In [44]:
# your code here

### Factorisation of the embedding matrix

We can see that at this point the embedding matrix has $V \cdot H = 28996 \cdot 768 = 22.268.928$ parameters. That's as much as a fifth of the entire model! Let's try to do something about it. The [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) proposes to factorise the embedding matrix into the product of two smaller matrices. Thus, the embedding parameters will contain $V \cdot E + E \cdot H$ elements, which is much smaller than $V \cdot H$ if $H \gg E$. The authors choose $E = 128$, but nothing prevents us from taking any other value. For example, by choosing $H = 64$, we reduce the number of parameters by about 20M.

__Task 3 [1 pts].__ Write a wrapper class over the embedding layer that implements factorisation into two matrices, and fine-tune the factorised model. Note, both matrices can be initialised using SVD decomposition so that the initial approximation is good. This will save a great amount of time during training. With decomposition rank $H = 64$ you should get an F1 greater than 0.87.

In [ ]:
# your code here

### Knowledge distillation

Knowledge distillation is a learning paradigm in which the knowledge of a teacher model is distilled into a student model. The student can be an arbitrary smaller model that solves the same problem, but typically the student has the same architecture as the teacher. Two error functions are used in the distillation:
1. Standard cross-entropy.
1. A function specifying the distance between the distributions of teacher and student predictions. KL-divergence is most commonly used.

To ensure that the teacher's prediction distribution is not degenerate, a temperature greater than 1, such as 2 or 5, is added to softmax.   
__Important:__ when dividing logits by temperature, the gradient values decrease by a factor of $\tau^2$ (check this!). So to return them to their original scale, the error must be multiplied by $\tau^2$. You can read more about this in section 2.1 [of the original article](https://arxiv.org/pdf/1503.02531).

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png" width="1000">

__Task 4 [3 pts].__ Implement the knowledge distillation method shown in the picture. Use KL-divergence [`nn.KLDivLoss(reduction="batchmean")`](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html) (be careful with the format of its inputs) to calculate the loss between student and teacher predictions. Sum the soft loss with the hard loss to get the total loss.   
As the teacher, use the pre-trained BERT from task 2. As a student, take the untrained model with a size __no larger than 20M__ parameters. You can use factorisation of the embedding matrix to reduce the number of parameters. If you have done everything correctly, you should get an F1 value of at least 0.7 on the test set. You should be able to do this with about 20k iterations of training. If you fail, you can refer to [DistilBERT](https://arxiv.org/abs/1910.01108) and [this article](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) for details.

__Important:__
* Don't forget to add _warmup_ when training a student.
* Don't forget to put the teacher in _eval_ mode.

In [ ]:
# your code here

## Tasks to choose from

As you realise, there are still quite a few different ways to reduce the size of a trained model. This section asks you to implement different techniques. You can get a different number of points for each one depending on the difficulty. Successful implementation will be judged on both code and quality on a test set. Any points for this assignment that you score above 10 will be considered bonus points.  
In task 4, you trained a model with a 20M parameter constraint. When implementing the techniques in this section, stick to the same restriction. This will allow you to fairly compare the techniques against each other and draw the correct conclusions. Write in your report about everything you tried.

* __Weights sharing [2 pts].__ The [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) modification of BERT proposes to share weights between layers in addition to embedding factorisation. That is, different layers use the same weights. This technique is equivalent to applying the same layer several times. It allows to reduce the number of parameters several times and not to lose much in quality.
* __Factorisation of intermediate layers [2 pts].__ If you can factorise the embedding matrix, you can factorise everything else too. There are many different approaches for factorising layers and it is hard to choose one. You can be inspired by [this list](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), find something else on the internet, or come up with a method on your own. Either way, in the report, justify why you decided to do it the way you did.
* __Distillation for intermediate layers (2 points).__ We discussed that in addition to minimizing the distance between the outputs of the student and the teacher, you can minimize the distance between their intermediate layers. [This paper](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) details how this can be done.
* __Pruning (4 points).__ The [SparseGPT](https://arxiv.org/abs/2301.00774) method proposes an approach that removes some of the model weights after training. Turns out that it is possible to remove up to half of all weights without loss in quality. The maths behind the method is quite complex, but the general approach is simple - we will remove weights in each layer separately and when removing part of the layer weights, the remaining weights will be reconfigured so that the overall output of the layer is unchanged.
* __Removal of heads (6 points).__ At this point we are using all attention heads, but a number of studies show that most of them can be discarded without loss of quality. This [paper](https://arxiv.org/pdf/1905.09418.pdf) proposes an approach that adds gates to the attention mechanism that regulate which heads participate in the layer and which do not. During training, the gates are adjusted so that most heads are not used. At the end of training, unused heads can be removed. A lot of points are given for this task because the method has quite complex math and it is hard to make the approach work. If you decide to spend your effort on it, we will give intermediate points based on the report if you fail.   
__Tip:__ while training a model, watch the behaviour of the gates carefully. If you have done everything correctly, they should zero out. However, they do not always zero out immediately, you should give them time and train the model longer.